# Intro. to Context Aware Snorkel: Extracting Spouse Relations from the News Across Sentences

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [7]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      10,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=40900  #epochs=10  batch size=64
[LSTM] Epoch 1 (170.58s)	Average loss=0.622903	Dev F1=14.80
[LSTM] Epoch 2 (351.62s)	Average loss=0.614500	Dev F1=6.62
[LSTM] Epoch 3 (532.80s)	Average loss=0.613445	Dev F1=14.58
[LSTM] Epoch 4 (722.42s)	Average loss=0.612276	Dev F1=18.97
[LSTM] Epoch 5 (905.92s)	Average loss=0.612186	Dev F1=13.60
[LSTM] Epoch 6 (1085.37s)	Average loss=0.612291	Dev F1=0.00
[LSTM] Epoch 7 (1261.88s)	Average loss=0.611218	Dev F1=15.01
[LSTM] Epoch 8 (1433.43s)	Average loss=0.610519	Dev F1=17.41
[LSTM] Epoch 9 (1608.46s)	Average loss=0.609914	Dev F1=15.00
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 10 (1790.99s)	Average loss=0.609425	Dev F1=15.85
[LSTM] Model saved as <LSTM>
[LSTM] Training done (1802.81s)
[LSTM] Loaded model <LSTM>


Now, we get the precision, recall, and F1 score from the discriminative model:

In [8]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.107, Recall: 0.693, F1 Score: 0.185


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [9]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.693
Neg. class accuracy: 0.805
Precision            0.107
Recall               0.693
F1                   0.185
----------------------------------------
TP: 151 | FP: 1260 | TN: 5206 | FN: 67



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [10]:
lstm.save_marginals(session, test_cands)

Saved 6684 marginals


##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!